In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1단계: PDF 문서 로드 및 분할
pdf_path = "../data/콘텐츠분쟁해결_사례.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()  # 페이지별 Document 리스트

# 텍스트 분할기 설정 (법률 사례 최적화)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,         #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,       #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",       #  법률 문서 섹션 구분자
        "\n【쟁점사항】",       #  쟁점 부분 구분
        "\n【처리경위】",       #  처리 과정 구분
        "\n【처리결과】",       #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

texts = text_splitter.split_documents(documents)  # 문서 분할 후 리스트

print(f"총 분할된 문서 조각 수: {len(texts)}")  # 확인용 출력


총 분할된 문서 조각 수: 104


In [ ]:

# 2단계: 임베딩 모델 설정
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.12\Lib\site-packages\langchain_community\embeddings\openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


In [14]:

# 3단계: 벡터 저장소 생성 및 검색기 설정
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [ ]:

# 4단계: LLM 설정
llm = ChatOpenAI(
    model="gpt-4o",         #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,        #  법률 조언은 정확성 중시
    max_tokens=2000         #  충분한 답변 길이
)


In [ ]:

# 5단계: 법률 자문 프롬프트 작성
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [ ]:

# 6단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                                     #  위에서 설정한 LLM
    chain_type="stuff",                          #  문서들을 하나로 합쳐서 처리
    retriever=retriever,                         #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},        #  법률 자문 프롬프트
    return_source_documents=True                 #  참조 문서도 함께 반환
)


In [ ]:

# 7단계:  테스트 질문 작성성
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]



In [ ]:

# 8단계: 테스트 질문 예시 및 실행
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

for question in test_questions:
    dispute_type = classify_dispute_type(question)
    print(f"\n분쟁 유형: {dispute_type}")
    print(f"질문: {question}")
    result = qa_chain({"query": question})  # 키는 'query' 또는 'question'으로 테스트
    print(f"답변: {result['result']}\n")
    
    print("참조 문서 예시:")
    for doc in result['source_documents'][:2]:  # 상위 2개 문서만 출력
        print(doc.page_content[:300], "...\n")



분쟁 유형: 게임
질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
답변: 온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률적 조언을 드리겠습니다. 제시된 사례들을 근거로 다음과 같이 답변드립니다.

1. **사례 분석**:
   - **사례 1 (2006년)**: 시스템 오류로 아이템이 소멸된 경우, 계정 명의자가 아닌 신청인의 복구 요청은 거부되었습니다. 계정 명의자가 직접 복구 신청을 해야 한다는 점이 강조되었습니다.
   - **사례 2 (2009년)**: 시스템 오류로 인한 게임머니 소실 주장에 대해, 시스템 오류가 발생하지 않았다는 게임사의 입장이 있었습니다. 신청인의 통신회선 문제일 가능성이 제기되었습니다.
   - **사례 3 (2009년)**: 해킹으로 인한 아이템 손실의 경우, 보안서비스 미사용으로 인해 복구가 거부되었습니다.
   - **사례 4 (2007년)**: 해킹 아이템 회수와 관련하여, 게임머니 환급으로 신청인의 피해가 없다고 판단되었습니다.
   - **사례 5 (2008년)**: 보안서비스 미작동으로 인한 아이템 도난의 경우, 게임사가 도난을 인정하고 아이템을 복구해 주었습니다.

2. **관련 법령 및 조항**:
   - 민법 제250조 및 제251조는 도품이나 유실물에 대한 반환 청구에 관한 규정을 두고 있습니다. 그러나 이는 주로 물리적 재산에 대한 규정이며, 디지털 아이템에 직접 적용되지는 않습니다.

3. **실무적 해결방안**:
   - **계정 명의 확인**: 계정 명의자가 본인인지 확인하고, 명의자가 아닌 경우 명의자에게 복구 신청을 요청합니다.
   - **증거 수집**: 시스템 오류로 인한 아이템 소실을 입증할 수 있는 증거(스크린샷, 로그 기록 등)를 수집합니다.
   - **게임사와의 협의**: 게임사의 고객센터에 공식적으로 이의를 제기하고, 시스템 오류에 대한 구체적인 설명과 복